## IBGE Data Analysis Project

IBGE is the main source for all kinds os statistical data in Brazil. This repository is an exploration of interesting socioeconomic data as well as concepts like Data visualization, REST API abstraction and Test driven development applied to Data Science.

In [23]:
import urllib3
#import json
import matplotlib.pyplot as plot
import numpy
import pandas
from ibge_lib import *


In [24]:
http = urllib3.PoolManager()
http_fn = lambda url: http.request('GET', url)

aggrs_query_url = aggregates_url()
researches = query_ibge(aggrs_query_url, http_fn)

 ### Picking an aggregate

 IBGE's API needs the exact data aggregate code in order to supply any useful data. Therefore, we need to ask for the list of aggregates and search for the code of the aggregate we want. Luckly, these are grouped by research. In this case, we want the National Household Sampling Research (PNAD).

In [25]:
format_fn = lambda x: f"{x['id']}: {x['nome']}"
#list(map(format_fn, researches))

In [26]:
pnad_id = 'B5'
filter_pnad_fn = lambda x: x['id'] == pnad_id
pnad_data = next(filter(filter_pnad_fn, researches)) # 'next' gets the first element, in this case
#pnad_data

In [27]:
filtered_aggr = filter_aggregates(pnad_data, 'rendimento domiciliar per capita')
format_fn = lambda aggr: f"{aggr['id']}: {aggr['nome']}"
#list(map(format_fn, filtered_aggr))

In [28]:
income_upper_bounds_aggr = 7438
cumm_income_dist_aggr = 7530
avg_income_per_class_aggr = 7532

In [29]:
upper_bounds_metadata = query_ibge( metadata_url(income_upper_bounds_aggr), http_fn )
cumm_dist_metadata = query_ibge( metadata_url(cumm_income_dist_aggr), http_fn )
income_per_class_metadata = query_ibge( metadata_url(avg_income_per_class_aggr), http_fn )

In [30]:
#upper_bounds_metadata

In [31]:
upper_boounds_var = '10769'
upper_bounds_classif = '1019'
uppper_bounds_classes = [ str(x['id']) for x in upper_bounds_metadata['classificacoes'][0]['categorias'] ]
uppper_bounds_classes_fmt = f'[{",".join(uppper_bounds_classes)}]'
#uppper_bounds_classes_fmt

In [35]:
var_query_url = variables_url( income_upper_bounds_aggr, [upper_boounds_var], [f'{upper_bounds_classif}{uppper_bounds_classes_fmt}'])
upper_bounds_response = query_ibge(var_query_url, http_fn )
# upper_bounds_response

In [41]:
income_percentiles = [5, 10, 20, 30, 40, 50, 60, 70, 80, 90, 95, 99]
income_upper_bounds_results = upper_bounds_response[0]['resultados'][1:]
#income_upper_bounds_results
years_queried = list(income_upper_bounds_results[0]['series'][0]['serie'].keys())
years_queried


['2015', '2016', '2017', '2018', '2019', '2020']

In [34]:
X = income_percentiles
widths = numpy.array([-5,  -5, -10, -10, -10, -10, -10, -10, -10, - 10,  -5,  -4,   -1])
Y = numpy.array(income_dist)/(-widths)

fig, ax = plot.subplots()
ax.bar(X, Y, align='edge', width=widths)

NameError: name 'income_dist' is not defined